<a href="https://colab.research.google.com/github/kgoz12/JSL/blob/master/Error_saving_pipeline_containing_drug_normalizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/23.Drug_Normalizer.ipynb)

Save pipeline model containing drug normalizer.

In [ ]:
from google.colab import drive

In [ ]:
# !ls "/content/drive/MyDrive/new_models"

In [ ]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving jsl_keys.json to jsl_keys (2).json


In [ ]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

In [ ]:
import json
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print (sparknlp.version())
print (sparknlp_jsl.version())

3.0.1
3.0.1


In [ ]:
 # if you want to start the session with custom params as in start function above

def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version)  \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")
      
    return builder.getOrCreate()

# spark = start(secret)

In [ ]:
spark

In [ ]:
# Sample data
data_to_normalize = spark.createDataFrame([
            ("A", "Sodium Chloride/Potassium Chloride 13bag", "Sodium Chloride / Potassium Chloride 13 bag"),
            ("B", "interferon alfa-2b 10 million unit ( 1 ml ) injec", "interferon alfa - 2b 10000000 unt ( 1 ml ) injection"),
            ("C", "aspirin 10 meq/ 5 ml oral sol", "aspirin 2 meq/ml oral solution")
        ]).toDF("cuid", "text", "target_normalized_text")

data_to_normalize.show(truncate=100)

+----+-------------------------------------------------+----------------------------------------------------+
|cuid|                                             text|                              target_normalized_text|
+----+-------------------------------------------------+----------------------------------------------------+
|   A|         Sodium Chloride/Potassium Chloride 13bag|         Sodium Chloride / Potassium Chloride 13 bag|
|   B|interferon alfa-2b 10 million unit ( 1 ml ) injec|interferon alfa - 2b 10000000 unt ( 1 ml ) injection|
|   C|                    aspirin 10 meq/ 5 ml oral sol|                      aspirin 2 meq/ml oral solution|
+----+-------------------------------------------------+----------------------------------------------------+



In [ ]:
# Annotator that transforms a text column from dataframe into normalized text (with all policy)

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

drug_normalizer = DrugNormalizer() \
    .setInputCols("document") \
    .setOutputCol("document_normalized") \
    .setPolicy("all")

drug_normalizer_pipeline = Pipeline(stages=[
    document_assembler,
     drug_normalizer
    ])

myPipeline = drug_normalizer_pipeline.fit(data_to_normalize)

In [ ]:
print(myPipeline)

PipelineModel_3c0378bc7090


In [ ]:
# Save a copy of the model
myPipeline.write().save(path = "/content/drive/MyDrive/brand_new_models")

Py4JJavaError: ignored

In [ ]:
# !ls "/content/drive/MyDrive/new_models"

In [ ]:
# load saved model
# pipeMod = PipelineModel.load("/content/drive/MyDrive/new_models")
